In [3]:
import numpy as np
import sounddevice as sd
import subprocess
import librosa
import tensorflow as tf

In [5]:

# Charger le modèle TFLite
interpreter = tf.lite.Interpreter(model_path='all_targets.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Définir les paramètres du traitement du signal
sample_rate = 8000  # Définir la fréquence d'échantillonnage appropriée
audio_duration = 1  # Durée de l'audio en secondes
num_mfcc = 16  # Nombre de coefficients MFCC à extraire

# Variable pour suivre l'état de la détection du son
sound_detected = False

# Fonction pour calculer les coefficients MFCC à partir du signal audio
def calc_mfcc_from_audio(signal):
    # Stocker les données audio dans le fichier time_series_1s.dat
    np.savetxt('./time_series_1s.dat', signal, fmt='%d', delimiter='\n')
    
    # Appeler la fonction C++ pour calculer les MFCC à partir de la série temporelle
    subprocess.run(["./compute_mfcc"], shell=True)
    
    # Charger les MFCC depuis le fichier mfcc_from_time_series.dat
    mfcc_from_time_series = np.genfromtxt('mfcc_from_time_series_1s.dat', delimiter='\n', dtype=None)
    mfcc_from_time_series = mfcc_from_time_series.reshape(num_mfcc, num_mfcc)
    
    return mfcc_from_time_series

# Fonction pour effectuer une prédiction avec le modèle TFLite
def predict(input_data):
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output_data)
    return predicted_class

# Fonction de rappel pour enregistrer l'audio à partir du microphone
def audio_callback(indata, frames, time, status):
    global sound_detected
    if status:
        print('Erreur de l\'enregistrement audio:', status)
    if not sound_detected:
        if np.max(np.abs(indata)) > 0.01:  # Détection du son (ajuster la valeur seuil selon votre environnement)
            sound_detected = True

    else:
        processed_data = calc_mfcc_from_audio(indata[:, 0])  # Prendre uniquement le canal gauche si l'audio est stéréo
        predicted_class = predict(processed_data)
        print('Classe prédite:', predicted_class)
        sound_detected = False

# Enregistrer l'audio à partir du microphone et effectuer la prédiction
with sd.InputStream(callback=audio_callback, channels=1, samplerate=sample_rate):
    print('En attente de détection du son...')
    while not sound_detected:
        pass
    print('Son détecté ! Enregistrement en cours...')
    sd.sleep(int(audio_duration * 1000))


En attente de détection du son...
Son détecté !
Son détecté ! Enregistrement en cours...
